<a href="https://colab.research.google.com/github/MohanPadala/gen-ai-project/blob/main/AnimeMovieRecommendSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchain_groq chromadb gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 6.4 MB/s eta 0:0

In [2]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq

In [6]:
anime = pd.read_csv("/content/anime_with_synopsis.csv")
anime.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [7]:
anime.dropna()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
264,296,Dragon Drive,6.73,"Action, Sci-Fi, Adventure, Comedy, Fantasy, Sc...","If there's one word to describe Reiji Ozora, i..."
265,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",ushuna is a blonde and very beautiful Senshi (...
266,298,.hack//Tasogare no Udewa Densetsu,6.60,"Adventure, Comedy, Fantasy, Game, Sci-Fi, Shounen","g the legendary characters ""Kite"" and ""Black R..."
267,299,.hack//Liminality,6.61,"Game, Mystery, Sci-Fi","hile playing the newly released MMORPG ""The Wo..."


In [8]:
anime['combined_df'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)


In [9]:
anime['combined_df'][0]

'Title: Cowboy Bebop. Overview: In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member\'s dark and mysterious past little by little. Well-balanced with high density action and light-hearted comedy, Cowboy Bebo

In [10]:
anime[['combined_df']].to_csv('anime_updated.csv', index = False)

In [11]:
pd.read_csv("/content/anime_updated.csv")

,combined_df
0,Title: Cowboy Bebop. Overview: In the year 207...
1,Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,Title: Trigun. Overview: Vash the Stampede is ...
3,Title: Witch Hunter Robin. Overview: ches are ...
4,Title: Bouken Ou Beet. Overview: It is the dar...
...,...
264,Title: Dragon Drive. Overview: If there's one ...
265,Title: Grenadier: Hohoemi no Senshi. Overview:...
266,Title: .hack//Tasogare no Udewa Densetsu. Over...
267,Title: .hack//Liminality. Overview: hile playi...


**Data** Loader and VectorStore using **Langchain**

In [24]:
loader=CSVLoader(file_path="/content/anime_updated.csv")
data = loader.load()

text_splitter=CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)
embeddings = HuggingFaceEmbeddings()
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile",
    temperature=0
)






docssearch = Chroma.from_documents(texts, embeddings)

/tmp/ipython-input-24-2479244635.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

**Querying Vector DB Store for movie recommendation**

In [13]:
query = "Iam looking for an animated sci-fi movie. What could you suggest movie"
docs=docssearch.similarity_search(query,k = 1)

In [14]:
docs

[Document(metadata={'source': '/content/anime_updated.csv', 'row': 161}, page_content='combined_df: Title: Xenosaga The Animation. Overview: The year is T.C. 4767. Four thousand years have passed since humanity abandoned its birthplace, the planet Earth. Beset by the hostile alien Gnosis, mankind is now scrambling to find ways to defeat this threat to their existence. The development of KOS-MOS (a specialized android with amazing capabilities) by Vector engineer Shion Uzuki was one response to the threat. But when their ship is destroyed by the Gnosis, Shion and her companions find themselves thrust into the middle of a battle with no clear sides... (Source: Anime-Planet) Genres: Action, Mecha, Sci-Fi, Shounen, Space')]

**Using QA Retrieval for movie recommendation**

In [15]:
qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=docssearch.as_retriever(),
    return_source_documents=True
)


In [16]:
query = "Can you recommend a romance anime with a unique storyline?"
result=qa({"query":query})

/tmp/ipython-input-16-3819930326.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query":query})


In [17]:
result['result']

'Based on the information provided, I would recommend "Loveless" as a romance anime with a unique storyline. The anime features a world where people are born with cat ears and tails that disappear after they engage in sexual intercourse, symbolizing virginity and innocence. The story follows a 12-year-old boy named Ritsuka Aoyagi and his partner Agatsuma Soubi as they navigate a complex world of fighting pairs and uncover the truth behind Ritsuka\'s brother\'s death. The anime combines elements of action, mystery, supernatural, and romance, making it a unique and intriguing watch.'

In [18]:
result['source_documents'][0]

Document(metadata={'row': 125, 'source': '/content/anime_updated.csv'}, page_content="combined_df: Title: Kimi ga Nozomu Eien. Overview: In the warmth of a golden sun, timid high school student Haruka Suzumiya confesses to Takayuki Narumi, the boy she admires. Takayuki accepts Haruka's confession and the two become a couple, supported by their mutual friends Mitsuki Hayase and Shinji Taira. All is well for these friends—until tragedy strikes. One of them is sent to an unenviable fate, lying dormant in a coma. Three years later, they awaken, but their sudden reappearance in everyone's lives threatens to test the limits of these friends' bonds. Genres: Drama, Romance, Slice of Life")

**Prompt Engineering**

In [19]:
from langchain.prompts import PromptTemplate

template = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end. For each question, suggest three anime, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context","question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=docssearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

query = "Give me a list of comedy animes that are lighthearted and fun."
result=qa({"query":query})
print(result['result'])


Here are three comedy anime recommendations that are lighthearted and fun:

1. **Happy☆Lesson (TV)**: This anime is a comedy, drama, and romance series that follows the story of Hitotose Chitose, a boy who lives with five female teachers who become his mothers. The show is a heartwarming and hilarious exploration of their daily lives, relationships, and misadventures. You might enjoy it because of its lighthearted and comedic take on family and relationships.
2. **Genshiken**: This anime is a slice-of-life comedy that explores the world of otaku culture through the eyes of Kanji Sasahara, a college freshman who joins a club for fans of anime and manga. The show is a humorous and relatable portrayal of fandom, friendship, and self-discovery. You might enjoy it because of its witty humor and lighthearted take on geek culture.
3. **Aa! Megami-sama! (TV)**: This anime is a supernatural comedy romance that follows the story of Keiichi Morisato, a human who accidentally summons a goddess nam

**Second Template**

In [20]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a movie recommender system that helps users find anime that match their preferences. Use the following pieces of context to answer the question at the end. For each question, take into account the context and the personal information provided by the user. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
"""

user_info = """This is what we know about the user, and you can use this information to better tune your research: Age: {age} Gender: {gender}
"""

template_suffix = """Question: {question}
Your response:"""

# Format user_info
user_info_filled = user_info.format(age=18, gender="female")

# Combine all parts
COMBINED_PROMPT = template_prefix + '\n' + user_info_filled + '\n' + template_suffix

print(COMBINED_PROMPT)


You are a movie recommender system that helps users find anime that match their preferences. Use the following pieces of context to answer the question at the end. For each question, take into account the context and the personal information provided by the user. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}

This is what we know about the user, and you can use this information to better tune your research: Age: 18 Gender: female

Question: {question}
Your response:


In [21]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import gradio as gr

# Define prompt
template_prefix = """You are a movie recommender system that helps users find anime that match their preferences. Use the following pieces of context to answer the question at the end. For each question, take into account the context and the personal information provided by the user. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
"""

user_info = """This is what we know about the user, and you can use this information to better tune your research: Age: {age} Gender: {gender}
"""

template_suffix = """Question: {question}
Your response:"""

user_info_filled = user_info.format(age=18, gender="female")
COMBINED_PROMPT = template_prefix + '\n' + user_info_filled + '\n' + template_suffix

PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=docssearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    result = qa({"query": query})
    return result['result']

# Gradio interface
iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme="Respair/Shiki@1.2.1"
)

if __name__ == "__main__":
    iface.launch()


theme_schema%401.2.1.json: 0.00B [00:00, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3b3c36dd9eb2dc832.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
